In [5]:
## Library
from IPython.display import display
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

In [6]:
## Load Data

transactions = pd.read_csv("../transactions.csv")
customers = pd.read_csv("../customers.csv")
products = pd.read_csv("../products.csv")

print("=== Data Loaded ===")
print(f"Transactions: {transactions.shape}")
print(f"Customers: {customers.shape}")
print(f"Products: {products.shape}\n")

=== Data Loaded ===
Transactions: (498, 5)
Customers: (100, 3)
Products: (50, 4)



In [7]:
## Ubah type data column timestamp pada transaction menjadi datetime
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'], errors='coerce')

In [8]:
## Exclude quantity yang null pada column transaction
transactions = transactions.dropna()

In [9]:
## Merged Data
merged = (
    transactions.merge(customers, on="customer_id", how="left")
    .merge(products, on="product_id", how="left")
)

In [10]:
## Buat column price dan month
merged["total_price"] = merged["quantity"] * merged["price"]
merged["month"] = merged["timestamp"].dt.to_period("M").astype(str)

In [11]:
merged.sample(5)

,transaction_id,customer_id,product_id,timestamp,quantity,join_date,customer_location,product_name,product_category,price,total_price,month
198,TRX0199,CUST049,PROD018,2025-04-01 14:00:00,1.0,2024-11-02,Bandung,Jaket Hoodie,Pakaian,400000,400000.0,2025-04
19,TRX0020,CUST014,PROD005,2023-08-20 10:40:00,5.0,2023-05-15,Jakarta,T-Shirt Katun,Pakaian,150000,750000.0,2023-08
76,TRX0077,CUST057,PROD037,2023-07-30 21:30:00,1.0,2023-07-25,Denpasar,Gaun Pesta,Pakaian,650000,650000.0,2023-07
113,TRX0114,CUST014,PROD016,2023-08-25 10:00:00,1.0,2023-05-15,Jakarta,Sprei Katun Jepang,Rumah Tangga,250000,250000.0,2023-08
51,TRX0052,CUST039,PROD009,2024-08-05 14:00:00,1.0,2024-06-01,Yogyakarta,Matras Yoga,Olahraga,300000,300000.0,2024-08


In [12]:
## Ingest to database

server = 'localhost'
database = 'ecommerce'
username = 'sa'               # user SQL Server
password = 'admin'  # password login SQL Server
table = 'analytic'

# Driver bawaan SQL Server (pastikan sudah terinstall ODBC)
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

In [13]:
merged.to_sql(
    name=table,
    con=engine,
    if_exists='replace',# bisa diganti 'append' kalau mau nambah data
    index=False
)

148

In [14]:
cek = """
SELECT COUNT(*) AS total_rows
FROM ecommerce.dbo.analytic
"""

df = pd.read_sql_query(cek, con=engine)
df.head()

,total_rows
0,496
